# Imports and preparation

In [1]:
import os

In [2]:
import numpy as np

In [3]:
import pandas as pd

In [4]:
from tqdm import tqdm

In [5]:
import time, gc

In [6]:
import importlib

In [7]:
from itertools import product


In [8]:
from util import util_elastic

OpenAI tiktoken module is not available for Python < 3.8,Linux ARM64 and AARCH64. Falling back to GPT2TokenizerFast.


In [9]:
from util import util_pipeline_v2 as util_pipeline

In [10]:
from util import util_experiment_juris_tcu as util_experiment

In [11]:
# Para ter repetibilidade nos resultados
random_state = 1

# Tratar valores infinitos (+ e -) como np.NaN
pd.options.mode.use_inf_as_na = True

# IMPORTANTE para tornar figuras interativas
# %matplotlib notebook

# Tamanho padrão das figuras
figsize=(10,6)

pd.set_option('display.max_row', 1000)

pd.set_option('display.max_columns', 50)

pd.set_option('display.column_space', 100)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', 200)


In [12]:
DATASET_NAME = 'juris_tcu'

In [13]:
PATH_QUERY = f'../data/{DATASET_NAME}/query.csv'
PATH_QREL =  f'../data/{DATASET_NAME}/qrel.csv'

# Data load

## Query data load

In [14]:
df_query = pd.read_csv(PATH_QUERY)

In [15]:
df_query.shape

(150, 3)

In [16]:
df_query.head()

,ID,TEXT,SOURCE
0,1,técnica e preço,search log
1,2,restos a pagar,search log
2,3,aditivo a contrato,search log
3,4,adesão a ata de registro de preços,search log
4,5,sobrepreço e superfaturamento,search log


In [17]:
df_query = df_query.reset_index(drop=True)

# Expand query

## Create reference to index (juris_tcu_index)

In [18]:
INDEX_NAME = 'indir_juris_tcu_index'

In [19]:
index = util_elastic.return_index(parm_index_name=INDEX_NAME)


Qtd de documentos 13255

Qtd de embeddings 13255

Documento.id=5: <Document: id=5, content='O termo é "Abono de permanência em serviço".
Abono de permanência em serviço tem definição: "O abono...'>


## Create pipeline

In [20]:
# Test query
# parm_query = "trata-se de uma denúncia contra o prefeito de Manhuaçu por não haver pago os funcionários da área de limpeza urbana"
parm_query = "despesa sem cobertura contratual"

In [23]:
pipes = []

In [21]:
ranker_type = 'MINILM_INDIR' # 'MT5_3B' # 'MT5_TRAINED_LIM50_800' # 'MINILM_TRAINED_19000'

In [56]:
FILTER_TERMO = {'class':['Termo']}

In [22]:
%%time
pipe_bm25_ranker = util_pipeline.return_pipeline_bm25_reranker(index, ranker_type, parm_limit_query_size=50)

Loading MINILM_INDIR with limit_query_size=50
CPU times: user 5.75 s, sys: 1.34 s, total: 7.09 s
Wall time: 4.88 s


In [27]:
util_pipeline

<module 'util.util_pipeline_v2' from '/home/borela/fontes/ind-ir/code/util/util_pipeline_v2.py'>

In [24]:
pipes.append({'PIPE_NAME': 'pipe_bm25_ranker',
              'PIPE_OBJECT': pipe_bm25_ranker,
              'RETRIEVER_TYPE': 'bm25',  
              'RETRIEVER_MODEL_NAME': '', 
              'RANKER_MODEL_NAME': util_pipeline.return_ranker_name(ranker_type)})

In [28]:
doctos_retornados_ranker = pipe_bm25_ranker.run(query=parm_query, params= {"Retriever":{"top_k":100, "filters": {"count_index_total": {"$gte": 5} }}})
util_pipeline.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Retriever': {'top_k': 100, 'filters': {'count_index_total': {'$gte': 5}}}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 100
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Custeio', '1102', 0.7551829218864441]
1 : ['Contratado', '1933', 0.7444576025009155]
2 : ['Contratação', '323', 0.7334334254264832]
3 : ['Contrato', '330', 0.7210038304328918]
4 : ['Pessoal', '1131', 0.7187904715538025]
5 : ['Rescisão contratual', '1392', 0.7135781645774841]
6 : ['Débito', '370', 0.712280809879303]
7 : ['Despesa', '421', 0.7094370126724243]
8 : ['Garantia contratual', '685', 0.7090778350830078]
9 : ['Despesa de custeio', '2282', 0.6935034394264221]
10 : ['Contrato administrativo', '331', 0.6914183497428894]


In [51]:
doctos_retornados_ranker = pipe_bm25_ranker.run(query=parm_query, params= {"Retriever":{"top_k":30, "filters": {"count_index_total": {"$gte": 5} }}})
util_pipeline.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Retriever': {'top_k': 30, 'filters': {'count_index_total': {'$gte': 5}}}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Custeio', '1102', 0.7552196979522705]
1 : ['Contrato', '330', 0.7209886312484741]
2 : ['Rescisão contratual', '1392', 0.7136325240135193]
3 : ['Débito', '370', 0.7123396396636963]
4 : ['Despesa', '421', 0.7094963192939758]
5 : ['Garantia contratual', '685', 0.7090460062026978]
6 : ['Despesa de custeio', '2282', 0.693473219871521]
7 : ['Despesa com pessoal', '422', 0.6910837292671204]
8 : ['Despesa de capital', '1100110', 0.6780967712402344]
9 : ['Despesa pública', '691', 0.672822892665863]
10 : ['Ordenação de despesa', '14780', 0.663760244846344]


In [53]:
doctos_retornados_ranker = pipe_bm25_ranker.run(query=parm_query, params= {"Retriever":{"top_k":30}})
util_pipeline.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Retriever': {'top_k': 30}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Gestão contratual', '1104190', 0.7272295951843262]
1 : ['Despesa sem prévio empenho', '1104589', 0.7147460579872131]
2 : ['Rescisão contratual', '1392', 0.7135682106018066]
3 : ['Despesa', '421', 0.7094001770019531]
4 : ['Garantia contratual', '685', 0.7090669274330139]
5 : ['Despesa de custeio', '2282', 0.6934110522270203]
6 : ['Despesa com pessoal', '422', 0.6909393668174744]
7 : ['Despesa sigilosa', '15088', 0.6875855326652527]
8 : ['Despesa de capital', '1100110', 0.6781857013702393]
9 : ['Despesa pública', '691', 0.6727723479270935]
10 : ['Despesa empenhada', '1104137', 0.6721410155296326]


In [57]:
doctos_retornados_ranker = pipe_bm25_ranker.run(query=parm_query, params= {"Retriever":{"top_k":50, "filters": FILTER_TERMO}})
util_pipeline.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Retriever': {'top_k': 50, 'filters': {'class': ['Termo']}}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 50
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Custeio', '1102', 0.7552196979522705]
1 : ['Gestão contratual', '1104190', 0.7272684574127197]
2 : ['Despesa sem prévio empenho', '1104589', 0.7146958112716675]
3 : ['Rescisão contratual', '1392', 0.7136325240135193]
4 : ['Débito', '370', 0.7123396396636963]
5 : ['Despesa', '421', 0.7094963192939758]
6 : ['Garantia contratual', '685', 0.7090460062026978]
7 : ['Despesa de custeio', '2282', 0.693473219871521]
8 : ['Despesa com pessoal', '422', 0.6910837292671204]
9 : ['Despesa sigilosa', '15088', 0.6876567602157593]
10 : ['Despesa de capital', '1100110', 0.6780967712402344]


### sts

In [32]:
%%time
pipe_join_bm25_sts_reranker = util_pipeline.return_pipeline_join_bm25_sts_reranker(index, ranker_type, parm_limit_query_size=50)

CPU times: user 3.16 s, sys: 593 ms, total: 3.76 s
Wall time: 2.06 s


In [33]:
pipes.append({'PIPE_NAME': 'pipe_join_bm25_sts_reranker',
              'PIPE_OBJECT': pipe_join_bm25_sts_reranker,
              'RETRIEVER_TYPE': 'join_sts_bm25',  
              'RETRIEVER_MODEL_NAME': '', 
              'RANKER_MODEL_NAME': util_pipeline.return_ranker_name(ranker_type)})

In [60]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 100}, "StsRetriever": {"top_k": 100}, "Ranker": {"top_k": 20}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker)

Parâmetros usados: {'Bm25Retriever': {'top_k': 100}, 'StsRetriever': {'top_k': 100}, 'Ranker': {'top_k': 20}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 20
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Custeio', '1102', 0.7551999688148499]
1 : ['Contratado', '1933', 0.7444170713424683]
2 : ['Gestão contratual', '1104190', 0.7272078990936279]
3 : ['Contrato', '330', 0.7210281491279602]
4 : ['Despesa sem prévio empenho', '1104589', 0.7147282958030701]
5 : ['Rescisão contratual', '1392', 0.713564395904541]
6 : ['Débito', '370', 0.7122265100479126]
7 : ['Despesa', '421', 0.709392249584198]
8 : ['Garantia contratual', '685', 0.7090484499931335]
9 : ['Despesa de custeio', '2282', 0.6934869289398193]
10 : ['Despesa com pessoal', '422', 0.6910235285758972]
CPU times: user 436 ms, sys: 24.9 ms, total: 461 ms
Wall time: 520 ms


In [61]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 50}, "StsRetriever": {"top_k": 100}, "Ranker": {"top_k": 30}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 30)

Parâmetros usados: {'Bm25Retriever': {'top_k': 50}, 'StsRetriever': {'top_k': 100}, 'Ranker': {'top_k': 30}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Custeio', '1102', 0.7552196979522705]
1 : ['Gestão contratual', '1104190', 0.7272684574127197]
2 : ['Despesa sem prévio empenho', '1104589', 0.7146958112716675]
3 : ['Rescisão contratual', '1392', 0.7136325240135193]
4 : ['Débito', '370', 0.7123396396636963]
5 : ['Despesa', '421', 0.7094963192939758]
6 : ['Garantia contratual', '685', 0.7090460062026978]
7 : ['Despesa de custeio', '2282', 0.693473219871521]
8 : ['Despesa com pessoal', '422', 0.6910837292671204]
9 : ['Despesa sigilosa', '15088', 0.6876567602157593]
10 : ['Passivo trabalhista', '15319', 0.6796528697013855]
11 : ['Despesa de capital', '1100110', 0.6780967712402344]
12 : ['Duplicata', '1100738', 0.6780412197113037]
13 : ['Passivo financeiro', '4506',

In [62]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 100}, "StsRetriever": {"top_k": 50}, "Ranker": {"top_k": 30}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 30)

Parâmetros usados: {'Bm25Retriever': {'top_k': 100}, 'StsRetriever': {'top_k': 50}, 'Ranker': {'top_k': 30}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Custeio', '1102', 0.7551999688148499]
1 : ['Contratado', '1933', 0.7444170713424683]
2 : ['Gestão contratual', '1104190', 0.7272078990936279]
3 : ['Contrato', '330', 0.7210281491279602]
4 : ['Despesa sem prévio empenho', '1104589', 0.7147282958030701]
5 : ['Rescisão contratual', '1392', 0.713564395904541]
6 : ['Débito', '370', 0.7122265100479126]
7 : ['Despesa', '421', 0.709392249584198]
8 : ['Garantia contratual', '685', 0.7090484499931335]
9 : ['Despesa de custeio', '2282', 0.6934869289398193]
10 : ['Despesa com pessoal', '422', 0.6910235285758972]
11 : ['Subempenho', '1108701', 0.6895459890365601]
12 : ['Despesa sigilosa', '15088', 0.6875990033149719]
13 : ['Despesa de capital', '1100110', 0.6782376170158386]


In [63]:
%%time
doctos_retornados_ranker = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 50}, "StsRetriever": {"top_k": 50}, "Ranker": {"top_k": 30}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doctos_retornados_ranker, 30)

Parâmetros usados: {'Bm25Retriever': {'top_k': 50}, 'StsRetriever': {'top_k': 50}, 'Ranker': {'top_k': 30}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Custeio', '1102', 0.7552024126052856]
1 : ['Gestão contratual', '1104190', 0.7272229194641113]
2 : ['Despesa sem prévio empenho', '1104589', 0.7147223949432373]
3 : ['Rescisão contratual', '1392', 0.7135782241821289]
4 : ['Débito', '370', 0.7123968005180359]
5 : ['Despesa', '421', 0.7094459533691406]
6 : ['Garantia contratual', '685', 0.709092378616333]
7 : ['Despesa de custeio', '2282', 0.6934863924980164]
8 : ['Despesa com pessoal', '422', 0.6909219026565552]
9 : ['Despesa sigilosa', '15088', 0.6876651048660278]
10 : ['Despesa de capital', '1100110', 0.678149938583374]
11 : ['Passivo financeiro', '4506', 0.6774552464485168]
12 : ['Despesa pública', '691', 0.6728728413581848]
13 : ['Despesa empenhada', '1104137',

In [64]:
%%time
doc_return = pipe_join_bm25_sts_reranker.run(query=parm_query, 
 params={"Bm25Retriever": {"top_k": 30}, "StsRetriever": {"top_k": 30}, "Ranker": {"top_k": 30}, "filters": FILTER_TERMO})
util_pipeline.detail_document_found(doc_return, 30)

Parâmetros usados: {'Bm25Retriever': {'top_k': 30}, 'StsRetriever': {'top_k': 30}, 'Ranker': {'top_k': 30}, 'filters': {'class': ['Termo']}}
Consulta: despesa sem cobertura contratual
Qtd documentos retornados: 30
Seguem os nomes dos termos recuperados em ordem de score
0 : ['Gestão contratual', '1104190', 0.7272229194641113]
1 : ['Despesa sem prévio empenho', '1104589', 0.7147223949432373]
2 : ['Rescisão contratual', '1392', 0.7135782241821289]
3 : ['Despesa', '421', 0.7094459533691406]
4 : ['Garantia contratual', '685', 0.709092378616333]
5 : ['Despesa de custeio', '2282', 0.6934863924980164]
6 : ['Despesa com pessoal', '422', 0.6909219026565552]
7 : ['Despesa sigilosa', '15088', 0.6876651048660278]
8 : ['Despesa de capital', '1100110', 0.678149938583374]
9 : ['Passivo financeiro', '4506', 0.6774552464485168]
10 : ['Despesa pública', '691', 0.6728728413581848]
11 : ['Despesa empenhada', '1104137', 0.6721331477165222]
12 : ['Seguro garantia', '1937', 0.6720420122146606]
13 : ['Cobertu

In [48]:
for docto in doc_return['documents'][:5]:
    print(docto.meta['name'])

Gestão contratual
Despesa sem prévio empenho
Rescisão contratual
Despesa
Garantia contratual


## Create expansion

In [100]:
from unidecode import unidecode
import string
import re

def normalize_text(text):
    # Remover acentos
    text_normalized = unidecode(text)
    # Remover espaços duplicados e espaços extras à direita e à esquerda
    text_normalized = re.sub('\s+', ' ', text_normalized).strip()
    # Remover pontuações
    text_normalized = ''.join(c for c in text_normalized.lower() if c not in string.punctuation)
    return text_normalized


In [105]:
from unidecode import unidecode

In [120]:
def set_normalize_text(text):
    # Remover espaços duplicados e espaços extras à direita e à esquerda
    text = re.sub(r'\s+', ' ', text.lower()).strip()
    # Remover acentos e caracteres especiais
    text = unidecode(text)
    # Separar as palavras em substrings
    words = text.split()
    # Retornar um conjunto de palavras normalizadas
    return set(words)

In [121]:
def test_similar_or_subtext(text1, text2):
    normalized_text1 = set_normalize_text(text1)
    normalized_text2 = set_normalize_text(text2)
    return normalized_text1 == normalized_text2 or normalized_text1.issubset(normalized_text2)


In [123]:
set_normalize_text('olá comO   vai voc? está bem.')

{'bem.', 'como', 'esta', 'ola', 'vai', 'voc?'}

In [129]:
test_similar_or_subtext("Licitação de técnica e preço","técnica e preço")

False

In [124]:
def expand_query(parm_pipe, parm_df_query, num_expansion_max):
    if num_expansion_max > 30:
        raise Exception('Review limit in code for num_expansion_max of 30!')
    count_expansion = [inteiro + 1 for inteiro in range(num_expansion_max)]
    for index, row in tqdm(parm_df_query.iterrows()):
        query_id = row['ID']
        doc_return = parm_pipe.run(query=row['TEXT'], params={"Bm25Retriever": {"top_k": 30}, "StsRetriever": {"top_k": 30}, "Ranker": {"top_k": num_expansion_max}, "filters": FILTER_TERMO})

        termos = []
        for docto in doc_return['documents'][:num_expansion_max * 2]:
            if not test_similar_or_subtext(docto.meta['name'], row['TEXT']): # and docto.meta['name'] not in termos:
                termos.append(docto.meta['name'])
            if len(termos) == num_expansion_max:
                break
        # print(len(termos), termos)

        for count in count_expansion:
            col_name = f"{count}"
            termos_concatenados = ', '.join(termos[:count]).rstrip(', ')
            parm_df_query.loc[index, col_name] = f"{row['TEXT']} - {termos_concatenados}"


In [125]:
df_query = pd.read_csv(PATH_QUERY)

In [126]:
%%time
expand_query(parm_pipe = pipe_join_bm25_sts_reranker, parm_df_query=df_query, num_expansion_max=10)

150it [00:23,  6.28it/s]

CPU times: user 23.2 s, sys: 305 ms, total: 23.5 s
Wall time: 23.9 s


In [127]:
df_query.head()

,ID,TEXT,SOURCE,1,2,3,4,5,6,7,8,9,10
0,1,técnica e preço,search log,técnica e preço - Licitação de técnica e preço,"técnica e preço - Licitação de técnica e preço, Proposta técnica","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço médio","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço médio, Preço de mercado","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço médio, Preço de mercado, Técnica jurídica","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço médio, Preço de mercado, Técnica jurídica, Proposta de preço","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço médio, Preço de mercado, Técnica jurídica, Proposta de preço, Preço máximo","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço médio, Preço de mercado, Técnica jurídica, Proposta de preço, Preço máximo, Preço estimado","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço médio, Preço de mercado, Técnica jurídica, Proposta de preço, Preço máximo, Preço estimado, Proposta","técnica e preço - Licitação de técnica e preço, Proposta técnica, Preço médio, Preço de mercado, Técnica jurídica, Proposta de preço, Preço máximo, Preço estimado, Proposta"
1,2,restos a pagar,search log,restos a pagar - Recolhimento,"restos a pagar - Recolhimento, Recompensa","restos a pagar - Recolhimento, Recompensa, Dívida","restos a pagar - Recolhimento, Recompensa, Dívida, Obrigação","restos a pagar - Recolhimento, Recompensa, Dívida, Obrigação, Nota de empenho de despesa","restos a pagar - Recolhimento, Recompensa, Dívida, Obrigação, Nota de empenho de despesa, Empenho","restos a pagar - Recolhimento, Recompensa, Dívida, Obrigação, Nota de empenho de despesa, Empenho, Recibado","restos a pagar - Recolhimento, Recompensa, Dívida, Obrigação, Nota de empenho de despesa, Empenho, Recibado, Pagamento indevido","restos a pagar - Recolhimento, Recompensa, Dívida, Obrigação, Nota de empenho de despesa, Empenho, Recibado, Pagamento indevido, Seguro de vida","restos a pagar - Recolhimento, Recompensa, Dívida, Obrigação, Nota de empenho de despesa, Empenho, Recibado, Pagamento indevido, Seguro de vida"
2,3,aditivo a contrato,search log,aditivo a contrato - Termo aditivo,"aditivo a contrato - Termo aditivo, Acréscimo","aditivo a contrato - Termo aditivo, Acréscimo, Reprovabilidade","aditivo a contrato - Termo aditivo, Acréscimo, Reprovabilidade, Atualização","aditivo a contrato - Termo aditivo, Acréscimo, Reprovabilidade, Atualização, Reformulação","aditivo a contrato - Termo aditivo, Acréscimo, Reprovabilidade, Atualização, Reformulação, Reenquadramento","aditivo a contrato - Termo aditivo, Acréscimo, Reprovabilidade, Atualização, Reformulação, Reenquadramento, Contrato administrativo","aditivo a contrato - Termo aditivo, Acréscimo, Reprovabilidade, Atualização, Reformulação, Reenquadramento, Contrato administrativo, Contrato de escopo","aditivo a contrato - Termo aditivo, Acréscimo, Reprovabilidade, Atualização, Reformulação, Reenquadramento, Contrato administrativo, Contrato de escopo","aditivo a contrato - Termo aditivo, Acréscimo, Reprovabilidade, Atualização, Reformulação, Reenquadramento, Contrato administrativo, Contrato de escopo"
3,4,adesão a ata de registro de preços,search log,adesão a ata de registro de preços - Sistema de Registro de Preços,"adesão a ata de registro de preços - Sistema de Registro de Preços, Ato sujeito a registro","adesão a ata de registro de preços - Sistema de Registro de Preços, Ato sujeito a registro, Intenção de Registro de Preços","adesão a ata de registro de preços - Sistema de Registro de Preços, Ato sujeito a registro, Intenção de Registro de Preços, Autoridade de registro","adesão a ata de registro de preços - Sistema de Registro de Preços, Ato sujeito a registro, Intenção de Registro de Preços, Autoridade de registro, Planilha de custos e formação de preços","adesão a ata de registro de preços - Sistema de Reg

In [ ]:
name_expansor = 'join_bm25_sts_30_minilm_indir'

In [ ]:
path_expansion = f'../data/juris_tcu_expanded_query/query_10_expansion_{name_expansor}.csv'

In [131]:
df_query.to_csv(path_expansion, index=False)